In [0]:
import pandas as pd

In [0]:
dicionario = pd.read_csv('artifcacts/dicionario.csv')
dicionario = dicionario[(dicionario.tabela != 'geolocation') & (dicionario.tabela != 'product_category_name_translation')]
display(dicionario)

In [0]:
%sql

select 
    count(distinct seller_id) as unique_selles,
    count(seller_id) as total_sellers 
from olist.default.sellers